https://ollama.com/  
https://github.com/ollama/ollama?tab=readme-ov-file  

1 - ollama 安装   
- https://github.com/ollama/ollama?tab=readme-ov-file
- https://github.com/ollama/ollama/blob/main/docs/linux.md  
- https://blog.csdn.net/u010197332/article/details/137604798?spm=1001.2014.3001.5506  

2 - ollama 配置大模型目录（Windows）  
- 打开 Windows 系统设置，来到【关于】，在右边找到【高级系统设置】，然后会出现一个【系统属性】窗口  
- 在上面的标签里选择【高级】，再点击下方的【环境变量】  
- 这时弹出一个【环境变量】窗口，点击【系统变量】下的新建，然后又弹出一个窗口  
- 变量名：OLLAMA_MODELS  
- 变量值：F:\LLM\ollama  

3 - ollama 配置大模型目录（Linux）  
- vim ~/.bashrc  
- export OLLAMA_MODELS=/home/Ollama/ollama_cache  
- cp -r /usr/share/ollama/.ollama/models /home/Ollama/ollama_cache  

4 - ollama 模型微调  
- https://blog.csdn.net/spiderwower/article/details/138755776?ops_request_misc=%257B%2522request%255Fid%2522%253A%25226fa4e7737d516cd7af7ea785e76a75a1%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=6fa4e7737d516cd7af7ea785e76a75a1&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_positive~default-1-138755776-null-null.142^v101^pc_search_result_base5&utm_term=ollama%20%E5%BE%AE%E8%B0%83&spm=1018.2226.3001.4187

4 - ollama 常见命令  
- ollama --version   
- ollama serve  
- ollama create /path/to/Modelfile  
- ollama show my_model  
- ollama run my_model  
- ollama list  
- ollama ps  
- ollama cp my_model new_model  
- ollama rm my_model  
- ollama --help  
- ollama run --help  

In [1]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import torch as th

from pprint import pp
from IPython.display import (Markdown, display)
from dotenv import load_dotenv
from openai import OpenAI
from ollama import Client

In [2]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [3]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [4]:
load_dotenv(dotenv_path="explore.env")
baidu_key = os.getenv("BAIDU_KEY")

## step-2: 实例化客户端

In [13]:
# 方法-1：Ollama
# client = Client(
#     host="http://127.0.0.1:11434",
#     timeout=60
# )

In [6]:
# 方法-2：OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1",
    api_key="EMPTY",
    timeout=60
    )

## step-3: 模型推理

In [7]:
checkpoint = "qwen2.5:7b"

In [8]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [9]:
user_prompt = "你好呀~"

In [10]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
gen_kwargs = {
    # "max_new_tokens": 512,
    # "do_sample": True,
    # "num_beams": 2,
    "temperature": 1.5,
    "top_p": 0.9,
}

# Ollama
# response = client.chat(
#     model=checkpoint,
#     messages=messages,
#     options=gen_kwargs,
#     stream=False
# )

# OpenAI
response = client.chat.completions.create(
    model=checkpoint,
    messages=messages,
    stream=False,
    **gen_kwargs
)

In [37]:
pp(response)

ChatCompletion(id='chatcmpl-488', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uxubapdr', function=Function(arguments='{"cityName":"南京"}', name='get_weather'), type='function', index=0)]))], created=1738240296, model='qwen2.5:7b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=21, prompt_tokens=226, total_tokens=247, completion_tokens_details=None, prompt_tokens_details=None))


In [18]:
# Ollama
# display(Markdown(response["message"]["content"]))
# OpenAI
display(Markdown(response.choices[0].message.content))

您好！很高兴为您服务。有任何差旅方面的问题都可以随时问我哦。

In [ ]:
# stream
response = client.chat.completions.create(
    model=checkpoint,
    messages=messages,
    stream=True,
    **gen_kwargs
)

for res in response:
    print(res.choices[0].delta.content)

## step-4: 工具调用

In [20]:
cityName2districtId = {
    "南京": "320100",
    "深圳": "440300"
}

In [21]:
def get_weather(cityName):
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

In [22]:
# test tool
data = get_weather(cityName="南京")
data

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 9, "feels_like": 8, "rh": 43, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u98ce", "uptime": "20250130202000"}, "forecasts": [{"text_day": "\\u591a\\u4e91", "text_night": "\\u9634", "high": 15, "low": 6, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u4e1c\\u98ce", "date": "2025-01-30", "week": "\\u661f\\u671f\\u56db"}, {"text_day": "\\u4e2d\\u96e8", "text_night": "\\u4e2d\\u96e8", "high": 9, "low": 7, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5317\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u5317\\u98ce", "date": "2025-01-31", "week": "\\u661f\\u671f\\u4e94"}, {"text_day": "\\u5c0f\\u96e8", "text_night": "\\u5c0f\\u96e8", "high": 9, "low": 4, "wc_day": "4~5\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~

In [23]:
get_weather_tool = {
    "name": "get_weather",
    "description": "根据输入的城市名称，查询天气",
    "parameters": {
        "type": "object",
        "properties": {
            "cityName": {
                "type": "string",
                "description": "城市名称"
            }
        },
        "required": ["cityName"]
    }
}

In [24]:
tools = [
    {
        "type": "function",
        "function": get_weather_tool
    }
]

In [25]:
tool_dict = {
    "get_weather": get_weather
}

In [26]:
user_prompt = "帮我查下南京的天气"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [27]:
# response = client.chat(
#     model=checkpoint,
#     messages=messages,
#     options=gen_kwargs,
#     stream=False,
#     tools=tools
# )

response = client.chat.completions.create(
    model=checkpoint,
    messages=messages,
    stream=False,
    tools=tools,
    **gen_kwargs
)

In [28]:
# assistant response
# assistant_response = response["message"]

assistant_response = response.choices[0].message
assistant_response.model_dump()


{'content': '',
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': [{'id': 'call_uxubapdr',
   'function': {'arguments': '{"cityName":"南京"}', 'name': 'get_weather'},
   'type': 'function',
   'index': 0}]}

In [29]:
messages.append(assistant_response)

In [ ]:
# tool response
# tool_name = assistant_response["tool_calls"][0]["function"]["name"]
# tool_to_call = tool_dict.get(tool_name)
# tool_args = assistant_response["tool_calls"][0]["function"]["arguments"]

tool_name = assistant_response.tool_calls[0].function.name
tool_to_call = tool_dict.get(tool_name)
tool_args = json.loads(assistant_response.tool_calls[0].function.arguments)

tool_response = tool_to_call(**tool_args)
tool_response

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 9, "feels_like": 8, "rh": 43, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u98ce", "uptime": "20250130202500"}, "forecasts": [{"text_day": "\\u591a\\u4e91", "text_night": "\\u9634", "high": 15, "low": 6, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u4e1c\\u98ce", "date": "2025-01-30", "week": "\\u661f\\u671f\\u56db"}, {"text_day": "\\u4e2d\\u96e8", "text_night": "\\u4e2d\\u96e8", "high": 9, "low": 7, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5317\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u5317\\u98ce", "date": "2025-01-31", "week": "\\u661f\\u671f\\u4e94"}, {"text_day": "\\u5c0f\\u96e8", "text_night": "\\u5c0f\\u96e8", "high": 9, "low": 4, "wc_day": "4~5\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~

In [31]:
messages.append(
    {
        "role": "tool",
        "content": tool_response,
        "tool_call_id": assistant_response.tool_calls[0].id
    }
)

In [32]:
messages

[{'role': 'system',
  'content': '你叫小慧助手，是由Lukas开发的差旅智能客服。你的身份是一名差旅秘书，你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。You are a helpful assistant on business travel.'},
 {'role': 'user', 'content': '帮我查下南京的天气'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uxubapdr', function=Function(arguments='{"cityName":"南京"}', name='get_weather'), type='function', index=0)]),
 {'role': 'tool',
  'content': '{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 9, "feels_like": 8, "rh": 43, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u98ce", "uptime": "20250130202500"}, "forecasts": [{"text_day": "\\u591a\\u4e91", "text_night": "\\u9634", "high": 15, "low": 6, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5

In [33]:
second_response = client.chat.completions.create(
    model=checkpoint,
    messages=messages,
    stream=False,
    **gen_kwargs
)

In [35]:
display(Markdown(second_response.choices[0].message.content))

当前南京的天气是多云，温度约为9°C。白天最高温度预计为15°C，夜晚最低温度约为6°C。风速在3到4级之间，白天主要来自东北西方向，晚上转为东北方向。

未来几天的天气预报如下：
- 2025年1月31日（周二），白天多云，夜晚晴朗，气温6至9°C。
- 2月1日（周三），全天晴朗，气温4至9°C。
- 2月2日（周四），全天多云转晴，气温3至9°C。
- 2月3日（周五），全天多云，气温-1至8°C。

请注意保暖和适时调整衣物。